# 案例: 基于设备数据的系统辨识

本脚本使用由`plant_simulation.py`（高保真设备仿真）生成的数据，来为一个渠道系统辨识一个简化的控制模型。具体来说，它将传感器数据（水位）与一个一阶加时延（FOPTD）模型进行拟合，该模型基于执行器数据（水闸开度）。

辨识出的参数（增益Kp、时间常数T、时延tau）可以用于设计控制器（例如PID控制器）。

### 1. 导入库并加载配置

In [ ]:
import json
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

CONFIG_PATH = 'examples/system_identification.json'
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

### 2. 定义FOPTD模型和辨识函数

In [ ]:
def foptd_model(t, Kp, T, tau):
    """定义一阶加时延 (FOPTD) 模型。"""
    response = np.zeros_like(t, dtype=float)
    T = max(T, 1e-9) # 避免数学错误
    mask = t >= tau
    response[mask] = Kp * (1 - np.exp(-(t[mask] - tau) / T))
    return response

def identify_model_parameters(cfg):
    """加载设备数据并执行系统辨识。"""
    print("\n--- 开始系统辨识 --- ")

    # 1. 加载数据
    input_file = cfg['data_source']['input_filename']
    try:
        data = pd.read_csv(input_file)
    except FileNotFoundError:
        print(f"错误: 未找到 `{input_file}`。")
        print("请先运行 `plant_simulation_zh.ipynb` 来生成数据。")
        return None, None
    print(f"成功加载 `{input_file}`。")

    # 2. 处理数据以寻找阶跃响应
    step_index = data['gate_opening'].diff().ne(0).idxmax()
    step_time = data['timestamp'][step_index]
    delta_u = data['gate_opening'][step_index] - data['gate_opening'][step_index - 1]
    y_initial = data['upstream_water_level'][step_index - 1]
    y_response = data['upstream_water_level'][step_index:] - y_initial
    t_response = data['timestamp'][step_index:] - step_time
    print(f"在t={step_time}s检测到大小为{delta_u:.2f}的阶跃输入。")

    # 3. 执行曲线拟合
    y_normalized = y_response / delta_u
    
    # 提供参数的初始猜测值
    fit_cfg = cfg['curve_fitting']
    Kp_guess = y_normalized.iloc[-1]
    T_guess = (t_response.iloc[-1] - t_response.iloc[0]) / fit_cfg['T_divisor']
    initial_guesses = [Kp_guess, T_guess, fit_cfg['tau']]
    print(f"初始猜测值: Kp={Kp_guess:.2f}, T={T_guess:.2f}, tau={fit_cfg['tau']:.2f}")

    try:
        params, _ = curve_fit(foptd_model, t_response.to_numpy(), y_normalized.to_numpy(), p0=initial_guesses, bounds=([-np.inf, 0, 0], [np.inf, np.inf, np.inf]))
    except Exception as e:
        print(f"曲线拟合出错: {e}")
        return None, None

    Kp_fit, T_fit, tau_fit = params
    print("\n--- 系统辨识结果 ---")
    print(f"辨识出的过程增益 (Kp): {Kp_fit:.4f} m / 单位闸门开度")
    print(f"辨识出的时间常数 (T): {T_fit:.4f} s")
    print(f"辨识出的时延 (tau): {tau_fit:.4f} s")
    return data, (Kp_fit, T_fit, tau_fit)


### 3. 运行辨识并可视化结果

In [ ]:
plant_data, fitted_params = identify_model_parameters(config)

if plant_data is not None and fitted_params is not None:
    Kp, T, tau = fitted_params
    
    # 重新计算阶跃响应以绘图
    step_index = plant_data['gate_opening'].diff().ne(0).idxmax()
    step_time = plant_data['timestamp'][step_index]
    delta_u = plant_data['gate_opening'][step_index] - plant_data['gate_opening'][step_index - 1]
    y_initial = plant_data['upstream_water_level'][step_index - 1]
    t_response = plant_data['timestamp'][step_index:] - step_time
    
    y_fitted_normalized = foptd_model(t_response, Kp, T, tau)
    y_fitted = y_fitted_normalized * delta_u + y_initial

    plt.figure(figsize=(12, 8))
    plt.plot(plant_data['timestamp'], plant_data['upstream_water_level'], 'b-', label='实际设备水位 (来自CSV)')
    plt.plot(t_response + step_time, y_fitted, 'r--', linewidth=2, label=f'拟合的FOPTD模型 (Kp={Kp:.2f}, T={T:.2f}, τ={tau:.2f})')
    plt.axvline(x=step_time, color='grey', linestyle=':', label=f'阶跃输入 (t={step_time}s)')
    plt.title('系统辨识: 设备数据 vs. 拟合的FOPTD模型')
    plt.xlabel('时间 (s)')
    plt.ylabel('上游水位 (m)')
    plt.legend()
    plt.grid(True)
    
    output_filename = config['output']['plot_filename']
    plt.savefig(output_filename)
    print(f"\n--- 结果图已保存至 {output_filename} ---")
    plt.show()